# Spark-ocr visual doc classifier lilt preprocessing of RVL-CDIP dataset

It is a sample of how to do preprocessing on databricks cluster with loading and saving data on s3 storage

In [ ]:
access_key = ""
secret_key = ""
encoded_secret_key = secret_key.replace("/", "%2F")
aws_bucket_name = ""
mount_name = "s3_dev"

try:
  dbutils.fs.mount("s3a://%s:%s@%s" % (access_key, encoded_secret_key, aws_bucket_name), "/mnt/%s" % mount_name)
except:
  dbutils.fs.unmount("/mnt/%s" % mount_name)
  dbutils.fs.mount("s3a://%s:%s@%s" % (access_key, encoded_secret_key, aws_bucket_name), "/mnt/%s" % mount_name)


In [0]:
%%bash
rm /dbfs/FileStore/johnsnowlabs/license.key

In [0]:
%sh echo "" >> /dbfs/FileStore/johnsnowlabs/license.key

In [ ]:
%%bash
cat /dbfs/FileStore/johnsnowlabs/license.key

In [ ]:
imagePath = "dbfs:/mnt/s3_dev/ocr/datasets/rvl_cdip_full_/*.tif"
df = spark.read.format("binaryFile").load(imagePath)
print(df.count())

In [0]:
import os

label_names = {0: "letter",
               1: "form",
               2: "email",
               3: "handwritten",
               4: "advertisement",
               5: "scientific_report",
               6: "scientific_publication",
               7: "specification",
               8: "file_folder",
               9: "news_article",
               10: "budget",
               11: "invoice",
               12: "presentation",
               13: "questionnaire",
               14: "resume",
               15: "memo"
}

files_labelled = {}
with open("/dbfs/mnt/s3_dev/ocr/datasets/rvl_cdip_train_labels.txt") as file:
    lines = file.readlines()
    for l in lines:
      l_ = l.strip().split(" ")
      head, tail = os.path.split(l_[0])
      #print(tail, label_names[int(l_[1])])
      files_labelled[tail] = label_names[int(l_[1])]

In [0]:
from pyspark.sql.functions import udf

def get_label(fl):
  head, fname = os.path.split(fl)
  if fname in files_labelled:
    return files_labelled[fname]
  else:
    print("File is missed:", fname)
    return None

get_label_udf = udf(get_label)

df = df.withColumn("act_label", get_label_udf("path"))
df.show(5)

+--------------------+-------------------+------+--------------------+-------------+
 path| modificationTime|length| content| act_label|
+--------------------+-------------------+------+--------------------+-------------+
dbfs:/mnt/s3_dev/...|2022-07-28 17:21:50|962306|[49 49 2A 00 2C A...|advertisement|
dbfs:/mnt/s3_dev/...|2022-07-29 10:06:47|934890|[49 49 2A 00 14 4...| null|
dbfs:/mnt/s3_dev/...|2022-07-27 15:51:57|909694|[49 49 2A 00 A8 D...|advertisement|
dbfs:/mnt/s3_dev/...|2022-07-30 10:16:04|907192|[49 49 2A 00 E2 D...| null|
dbfs:/mnt/s3_dev/...|2022-07-29 20:41:01|868244|[49 49 2A 00 BE 3...| invoice|
+--------------------+-------------------+------+--------------------+-------------+
only showing top 5 rows

In [ ]:
df = df.dropna(subset="act_label")
df.select("path", "act_label").show(truncate = False)

In [0]:
df = df.repartition(18732)
df.rdd.getNumPartitions()

Out[20]: 18732

In [ ]:
from sparkocr.transformers import *
from sparkocr.enums import *
from pyspark.ml import PipelineModel

binary_to_image = BinaryToImage()\
    .setOutputCol("image") \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

img_to_hocr = ImageToHocr()\
    .setInputCol("image")\
    .setOutputCol("hocr")\
    .setIgnoreResolution(False)\
    .setOcrParams(["preserve_interword_spaces=0"])

# OCR pipeline
pipeline1 = PipelineModel(stages=[
    binary_to_image,
    img_to_hocr,
])

df = pipeline1.transform(df).cache()
df = df.withColumnRenamed("image", "orig_image")
display(df)

In [ ]:
from sparkocr.utils import get_vocabulary_dict

vocab_file = "/dbfs/mnt/s3_dev/ocr/test_models/LiLT_vocabulary.txt"
vocab = get_vocabulary_dict(vocab_file, ",")

doc_class = VisualDocumentClassifierLilt() \
  .setInputCol("hocr") \
  .setOutputCol("label")
doc_class.setVocabulary(vocab)

result = doc_class.getPreprocessedDataset(
  df
  ).cache()

In [ ]:
result.select("path", "input_ids", "bbox", "attention_mask", "act_label").write.parquet("dbfs:/mnt/s3_dev/ocr/datasets/RVL-CDIP/processed_data_lilt")